In [50]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from shapely.geometry import Point
import pandana
import contextily as ctx

import warnings
warnings.filterwarnings('ignore')

import imageio
import os

# Set the variables for the current city

In [86]:
place = 'Rotterdam, the Netherlands'
minutes = 35
walk_speed = 85 # meter/minute
bike_speed = 250 # meter/minute = 15 km/h
drive_speed = 416 # meter/minute = 25 km/h

# Preparing the graphs (and GDFs) for walk, bike and drive networks

In [53]:
graph_walk = ox.graph_from_address(place,network_type='walk',dist=minutes*bike_speed)
graph_bike = ox.graph_from_address(place,network_type='bike',dist=minutes*bike_speed)
graph_drive = ox.graph_from_address(place,network_type='drive',dist=minutes*bike_speed)

In [54]:
nodes_walk,edges_walk =ox.utils_graph.graph_to_gdfs(graph_walk, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)
nodes_bike,edges_bike =ox.utils_graph.graph_to_gdfs(graph_bike, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)
nodes_drive,edges_drive =ox.utils_graph.graph_to_gdfs(graph_drive, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)


In [87]:
# Find the center nodes of the all 

minx = edges_walk.bounds.minx.min()
miny = edges_walk.bounds.miny.min()
maxy = edges_walk.bounds.maxy.max()
maxx = edges_walk.bounds.maxx.max()

center_node_walk = ox.get_nearest_node(graph_walk,((miny+maxy)/2,(minx+maxx)/2))
center_node_bike = ox.get_nearest_node(graph_bike,((miny+maxy)/2,(minx+maxx)/2))
center_node_drive = ox.get_nearest_node(graph_drive,((miny+maxy)/2,(minx+maxx)/2))

# Calculating the used for each minute

In [59]:
for u, v, k, data in graph_walk.edges(data=True, keys=True):
    data['time'] = data['length'] / walk_speed

for u, v, k, data in graph_bike.edges(data=True, keys=True):
    data['time'] = data['length'] / bike_speed

for u, v, k, data in graph_drive.edges(data=True, keys=True):
    data['time'] = data['length'] / drive_speed


### for each minute, creating a subgraph and saving only the new nodes (nodes that didn't appear in previous subgraphs)

In [88]:
times = list(range(minutes))
nodes_per_time_walk = []
previous_nodes = []
for time in times:
    subgraph = nx.ego_graph(graph_walk, center_node_walk, radius=time,  distance='time')
    current_nodes = list(subgraph.nodes)
    new_nodes = [x for x in current_nodes if x not in previous_nodes]
    nodes_per_time_walk.append(new_nodes)
    previous_nodes = current_nodes
    
nodes_per_time_bike = []
previous_nodes = []
for time in times:
    subgraph = nx.ego_graph(graph_bike, center_node_bike, radius=time,  distance='time')
    current_nodes = list(subgraph.nodes)
    new_nodes = [x for x in current_nodes if x not in previous_nodes]
    nodes_per_time_bike.append(new_nodes)
    previous_nodes = current_nodes
    
nodes_per_time_drive = []
previous_nodes = []
for time in times:
    subgraph = nx.ego_graph(graph_drive, center_node_drive, radius=time,  distance='time')
    current_nodes = list(subgraph.nodes)
    new_nodes = [x for x in current_nodes if x not in previous_nodes]
    nodes_per_time_drive.append(new_nodes)
    previous_nodes = current_nodes

# Creating an image for each minute

In [89]:
minx_3857 = edges_walk.to_crs('epsg:3857').bounds.minx.min()
miny_3857 = edges_walk.to_crs('epsg:3857').bounds.miny.min()
maxy_3857 = edges_walk.to_crs('epsg:3857').bounds.maxy.max()
maxx_3857 = edges_walk.to_crs('epsg:3857').bounds.maxx.max()


In [90]:
images = []
for current_nodes_walk,current_nodes_bike,current_nodes_drive,time in zip(nodes_per_time_walk,nodes_per_time_bike,nodes_per_time_drive,times):

    fig, ax = plt.subplots(figsize=(12,8))
    
    plt.ioff()

    if current_nodes_walk:
        nodes_walk[nodes_walk.index.isin(current_nodes_walk)].to_crs('epsg:3857').plot(ax=ax,color="green")
    if current_nodes_bike:
        nodes_bike[nodes_bike.index.isin(current_nodes_bike)].to_crs('epsg:3857').plot(ax=ax,color="blue")
    if current_nodes_drive:
        nodes_drive[nodes_drive.index.isin(current_nodes_drive)].to_crs('epsg:3857').plot(ax=ax,color="red")
    ax.set_xlim(minx_3857, maxx_3857)
    ax.set_ylim(miny_3857, maxy_3857)
    
    ctx.add_basemap(ax,url=ctx.providers.CartoDB.DarkMatter)#PositronNoLabels)
    
    current_image_name = place+str(time)+".jpg"
    images.append(current_image_name)
    
    plt.savefig(current_image_name)
    
    plt.close()

# Creating a GIF and deleting all temporary files

In [91]:

with imageio.get_writer(place+'.gif', mode='I') as writer:
    for f in images:
        image = imageio.imread(filename)
        writer.append_data(image)
writer.close()

In [92]:

for f in images:
    os.remove(f)
